# Machine Learning Using Julia

## Activate venv

In [1]:
using Pkg; Pkg.activate("julia-ml")
Pkg.status()

    Status `/media/halper/Seagate/projects/julia_iris_ml/julia-ml/Project.toml`
  [336ed68f] CSV v0.8.2
  [a93c6f00] DataFrames v0.21.8
  [7806a523] DecisionTree v0.10.10
  [38e38edf] GLM v1.3.11
  [b1bec4e5] LIBSVM v0.4.0
  [38d8eb38] Lathe v0.0.9
  [add582a8] MLJ v0.15.0
  [d491faf4] MLJModels v0.13.1
  [3646fa90] ScikitLearn v0.6.2
  [3eaba693] StatsModels v0.6.15


## Import dataset

In [2]:
using CSV, DataFrames

iris = DataFrame(CSV.File("../../backup_datasets/iris/iris.csv"))

species = iris[:, :species]

150-element PooledArrays.PooledArray{String,UInt32,1,Array{UInt32,1}}:
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 ⋮          
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

## Preprocessing

### One-hot encoding

In [ ]:
using Lathe

scaled_feature = Lathe.preprocess.OneHotEncode(iris, :species)  # Perform OH encoding

iris = select!(iris, Not([:species]))  # Remove original species column

first(iris, 5)


## Train/test-split

In [ ]:
using Random

sample = randsubseq(1:size(iris, 1), 0.75)
train = iris[sample, :]
notsample = [i for i in 1:size(iris, 1) if isempty(searchsorted(sample, i))]
test = iris[notsample, :]

y_test = species[notsample]

## Using GLM

In [ ]:
using DataFrames, GLM

fm_setosa = @formula(setosa ~ sepal_length + sepal_width + petal_length + petal_width)
lm_setosa = glm(fm_setosa, train, Binomial(), LogitLink())
pred_setosa = predict(lm_setosa, test)

fm_virginica = @formula(virginica ~ sepal_length + sepal_width + petal_length + petal_width)
lm_virginica = glm(fm_virginica, train, Binomial(), LogitLink())
pred_virginica = predict(lm_virginica, test)

fm_versicolor = @formula(versicolor ~ sepal_length + sepal_width + petal_length + petal_width)
lm_versicolor = glm(fm_versicolor, train, Binomial(), LogitLink())
pred_versicolor = predict(lm_versicolor, test)

preds = hcat(pred_setosa, pred_virginica, pred_versicolor)

In [ ]:
# Reclass by maximum predicted proba
preds_cat = String[];

for i in 1:nrow(DataFrame(preds))
    if pred_setosa[i] >= pred_virginica[i] && pred_setosa[i] >= pred_versicolor[i]
        preds_cat = vcat(preds_cat, "setosa")
    elseif pred_versicolor[i] >= pred_virginica[i] && pred_versicolor[i] >= pred_setosa[i]
        preds_cat = vcat(preds_cat, "versicolor")
    else
        preds_cat = vcat(preds_cat, "virginica")
    end
end

In [ ]:
# Compute accuracy of GLM

correct = 0

n = length(y_test)
for i in 1:n
    if y_test[i] == preds_cat[i]
        correct += 1
    end
end

println(correct / n)